In [66]:
from transformers import BertTokenizer, BertForTokenClassification, BertTokenizerFast
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification, pipeline
from sklearn.model_selection import train_test_split
import torch
import json
import evaluate
import numpy
from random import shuffle

In [56]:
label_list = ['B-LOC', 'I-LOC', 'O']
# 假设这是你的训练数据，以tokenized形式表示
with open('data_gen.json', 'r', encoding='utf-8') as f:
    datas = json.load(f)
  
new_datas = []
for idx, data in enumerate(datas):
    new_dict = {}
    new_dict['id'] = idx
    new_dict['tokens'] = data['text']
    new_dict['ner_tags'] = data['ner_tags']
    new_datas.append(new_dict)
    
# 转换成datasets库的格式
from datasets import Dataset
shuffle(new_datas)

train_dataset = Dataset.from_list(train_datas[len(new_datas) // 7 : ])
val_dataset = Dataset.from_list(train_datas[: len(new_datas)  // 7])
len(train_dataset), len(val_dataset)

(11672, 1945)

In [48]:
val_dataset

Dataset({
    features: ['text', 'address', 'ner_tags'],
    num_rows: 1945
})

In [49]:
a = train_dataset[1]
for i, j in zip(a['text'], a['ner_tags']):
    print(i, j)

我 0
家 0
住 0
在 0
岳 1
阳 2
市 2
君 2
山 2
区 2
君 2
山 2
街 2
道 2
1 2
2 2
3 2
号 2
， 0
这 0
附 0
近 0
有 0
个 0
垃 0
圾 0
桶 0
被 0
堵 0
住 0
了 0
， 0
希 0
望 0
能 0
尽 0
快 0
清 0
理 0
一 0
下 0
。 0


In [3]:

# 加载BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('google-bert/bert-base-chinese')

In [50]:

# 借助word_ids 实现标签映射
def process_function(datas):
    tokenized_datas = tokenizer(datas["text"], max_length=256, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(datas["ner_tags"]):
        word_ids = tokenized_datas.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_datas["labels"] = labels
    return tokenized_datas


new_train_datasets = train_dataset.map(process_function, batched=True)
new_val_datasets = val_dataset.map(process_function, batched=True)

Map:   0%|          | 0/11672 [00:00<?, ? examples/s]

Map:   0%|          | 0/1945 [00:00<?, ? examples/s]

In [51]:
tokenizer.decode(new_train_datasets[15]['input_ids']), new_train_datasets[15]['labels']

('[CLS] 我 家 在 中 山 市 南 区 某 小 区 ， 我 家 旁 边 的 垃 圾 桶 老 是 被 放 错 地 方 [SEP]',
 [-100,
  0,
  0,
  0,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100])

In [39]:

# 加载BERT模型，用于NER任务微调
model = BertForTokenClassification.from_pretrained('google-bert/bert-base-chinese',
                                                   num_labels=3)  ## 3

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
# step 4 评估函数：此处的评估函数可以从https://github.com/huggingface/evaluate下载到本地
seqeval = evaluate.load("./evaluate_/seqeval_metric.py")


def evaluate_function(prepredictions):
    predictions, labels = prepredictions
    predictions = numpy.argmax(predictions, axis=-1)
    # 将id转换为原始的字符串类型的标签
    true_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    result = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "f1": result["overall_f1"]
    }


In [54]:
# 训练设置
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=32,  # 训练时的batch_size
                               # gradient_checkpointing=True,     # *** 梯度检查点 ***
                               per_device_eval_batch_size=32,    # 验证时的batch_size
                               num_train_epochs=3,              # 训练轮数
                               logging_steps=20,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True      # 训练完成后加载最优模型
                               )

# 训练器
# step 6 创建Trainer
trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=new_train_datasets,
                  eval_dataset=new_val_datasets,
                  data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
                  compute_metrics=evaluate_function,
                  )

/root/miniconda3/lib/python3.8/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [57]:
# 开始训练
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.015300,0.023726,0.994593
2,0.014600,0.024385,0.994698
3,0.008600,0.023677,0.995086


TrainOutput(global_step=1095, training_loss=0.013654830874917714, metrics={'train_runtime': 70.9483, 'train_samples_per_second': 493.542, 'train_steps_per_second': 15.434, 'total_flos': 1084232115130896.0, 'train_loss': 0.013654830874917714, 'epoch': 3.0})

In [64]:
# step 8 模型评估
evaluate_result = trainer.evaluate(new_val_datasets)
print(evaluate_result)

{'eval_loss': 0.023677131161093712, 'eval_f1': 0.995086381414729, 'eval_runtime': 3.3167, 'eval_samples_per_second': 586.42, 'eval_steps_per_second': 18.392, 'epoch': 3.0}


In [72]:
# step 9：模型预测
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="simple")
res = ner_pipe("我家住在江苏省宿迁市宿城区，我有问题需要反应，北京国贸大厦有个地方洼陷，希望能求助")
print(res)

[{'entity_group': 'LABEL_0', 'score': 0.99999464, 'word': '我 家 住 在', 'start': 0, 'end': 4}, {'entity_group': 'LABEL_1', 'score': 0.758535, 'word': '江', 'start': 4, 'end': 5}, {'entity_group': 'LABEL_2', 'score': 0.9126012, 'word': '苏 省 宿 迁 市 宿 城 区', 'start': 5, 'end': 13}, {'entity_group': 'LABEL_0', 'score': 0.99999344, 'word': '， 我 有 问 题 需 要 反 应 ，', 'start': 13, 'end': 23}, {'entity_group': 'LABEL_2', 'score': 0.94298434, 'word': '北 京 国 贸 大 厦', 'start': 23, 'end': 29}, {'entity_group': 'LABEL_0', 'score': 0.99998933, 'word': '有 个 地 方 洼 陷 ， 希 望 能 求 助', 'start': 29, 'end': 41}]


In [74]:
tokenizer.save_pretrained('NER_model/')

('NER_model/tokenizer_config.json',
 'NER_model/special_tokens_map.json',
 'NER_model/vocab.txt',
 'NER_model/added_tokens.json',
 'NER_model/tokenizer.json')